# Experiments with `surprise`

This phase is to experiment with more classical models like SVD using surprise to create some baselines to compare my NCF model against.

## Generating recommendations for a user

I want to start off with just examining the quality of the recommendations. Just to get some feedback if things are going okay or not. I'll return to model analysis after.

In [4]:
# ankh
TEST_USER = 43708

Let's load the usual data back from our data store:

In [5]:
import pandas as pd
from collections import defaultdict
from surprise import Dataset, SVD, Reader
from surprise.model_selection import cross_validate
from surprise.model_selection import train_test_split

# Load wrangled data
store = pd.HDFStore('store.h5')
votes, vn = store['votes'], store['vn']

In [6]:
reader = Reader(rating_scale=(0, 1))
data = Dataset.load_from_df(votes, reader=reader)

I'll use SVD for now but later in the notebook I'll be comparing different models and running analysis.

In [7]:
trainset = data.build_full_trainset()
algo = SVD()
algo.fit(trainset)

We'll build a test-set by filtering out already read VNs from the list of all VNs. 

In [8]:
def build_anti_testset_for_user(algo, user_id):
    all_iids = vn.index.unique()
    iids_of_user = [iid for (iid, _) in algo.trainset.ur[user_id]]
    anti_testset = [(user_id, iid, 0) for iid in all_iids if iid not in iids_of_user]
    return anti_testset

anti_testset_user = build_anti_testset_for_user(algo, TEST_USER)
predictions = algo.test(anti_testset_user)

And voila, we can generate our predictions. A very important note here is to establish some sort of filtering procedure to weed out unpreferable VNs. Here, I'll use the following heuristic:
- Must be rated already and have over 50 votes minimum

I'll experiment with more finegrained procedures later, but this seems to give us some decent recommendations to start with.

In [9]:
# Recommendation generation
# Collect predictions and sort based on estimated rating
top_n = sorted([(pred.est, pred.iid) for pred in predictions], reverse=True)
# Compile dataframe to display recommendations
reccs = vn.loc[[x[1] for x in top_n]][['title', 'c_rating', 'c_votecount', 'tags']]
reccs['predicted'] = [x[0] for x in top_n]
reccs.loc[(reccs['c_votecount'] > 50) & ~(reccs['c_rating'].isna())].head(30)

,title,c_rating,c_votecount,tags,predicted
id,,,,,
2507,[Steal!],689.0,70,"[Multiple Endings, Protagonist with Voice Acti...",0.869705
20600,[真剣で私に恋しなさい！A-3],783.0,1121,"[Sex with Protagonist Only, Male Protagonist, ...",0.864820
8491,"[Angel, Devil, Elf and Me!, ¡Ángel, Demonio, E...",562.0,166,"[Male Protagonist, Unavoidable Harem Ending, N...",0.863529
7886,[アルバイトの先輩の女のコに仕事を教わっていたら始まっちゃうHな関係。],707.0,75,"[Nukige, High Sexual Content, Remote-controlle...",0.850756
2337,"[同級生2, 동급생 2]",767.0,65,"[90's, Nanpa, More Than Seven Endings]",0.849567
1458,[夢幻廻廊2 ～螺旋～],680.0,52,"[Time Loop, Master and Servant, Female Dominat...",0.848956
97,"[Saya no Uta, Saya no Uta ~ The Song of Saya, ...",801.0,16448,"[NVL, Madness, Horror]",0.847702
16563,[根雪の幻影 -白花荘の人々-],686.0,150,"[Haunted House, One True End, Ryokan]",0.840552
7304,[That Cheap and Sacred Thing],715.0,175,"[Kinetic Novel, Only a Single Hero, Female Pro...",0.826599


# Model analysis

There seems to be many models that `surprise` supports. I'm mainly interested in SVD, because that's what NCF will be building upon. 

Let's start off with some cross validation metrics.

In [10]:
cross_validate(algo, data, measures=["MSE"], cv=5, verbose=True)

Evaluating MSE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
MSE (testset)     0.0241  0.0240  0.0239  0.0240  0.0241  0.0240  0.0001  
Fit time          16.30   16.74   16.40   15.86   15.83   16.22   0.34    
Test time         2.62    2.22    2.09    2.12    2.00    2.21    0.22    


{'test_mse': array([0.02405904, 0.02402493, 0.02392401, 0.0240399 , 0.02407068]),
 'fit_time': (16.29642915725708,
  16.73988389968872,
  16.395029306411743,
  15.856948852539062,
  15.834071636199951),
 'test_time': (2.6249895095825195,
  2.2170963287353516,
  2.0936126708984375,
  2.1169626712799072,
  2.004941701889038)}